In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
from sklearn.model_selection import train_test_split

In [59]:
def load_data(in_file):
    SM_ML_df = pd.read_excel(in_file, sheet_name='SM_ML_values')
    reqd_data_arr = ['Plot_Id', 'Crop_Type', 'Field_SM', 'rsm_VATI', 'max_RSM', 'Soil_Class']
    SM_ML_df = SM_ML_df[reqd_data_arr]
    SM_ML_df.replace(0, np.nan, inplace=True)
    SM_ML_df.replace(-np.inf, np.nan, inplace=True)
    SM_ML_df = SM_ML_df.dropna(axis=0, how='any')
    return SM_ML_df

def replace_df_with_crop_classes_andMaptoInt(basepath, SM_ML_df):
    # Replacing with crop classes
    crop_class_file = basepath + 'd_crop_classes.xlsx'
    crop_class_df = pd.read_excel(crop_class_file, sheet_name='crop_classes')
    # print (crop_class_df)

    replacing_dict = {}
    for index, each_row in crop_class_df.iterrows():
    #     print (each_row, type(each_row))
        replacing_dict[each_row['Crop_Type']] = 'class' + str(each_row['Crop_Class'])
    # print (replacing_dict)

    SM_ML_df["Crop_Type"] = SM_ML_df["Crop_Type"].str.strip().replace(replacing_dict)
    # print (SM_ML_df)

    # Forming Dictionary for crop Type and replacing with integer in Crop Type Column
    Crop_Type_nparray = SM_ML_df['Crop_Type'].str.strip().unique()
    # print (Crop_Type_nparray)

    int_toCrop_type_dict = dict(enumerate(Crop_Type_nparray))
    crop_type_toInt_dict = {y:x for x,y in int_toCrop_type_dict.items()}
    # print (int_toCrop_type_dict)

    SM_ML_df["Crop_Type"] = SM_ML_df["Crop_Type"].str.strip().map(crop_type_toInt_dict)
#     print (SM_ML_df)
    return SM_ML_df

def make_data_ready(SM_ML_df):
    X = SM_ML_df[['Crop_Type', 'rsm_VATI', 'max_RSM', 'Soil_Class', 'Field_SM']]
    y = SM_ML_df[['Field_SM']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
#     X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values.flatten(), y_test.values.flatten()
    return (X_train, X_test, y_train, y_test)

def soil_mapping_function(X, k1, k2, s1):
    a,b = X
    return k1 + k2 * np.power(a, s1) / b

def plot_y_vs_y_predicted(coeff_k, coeff_s, X_test, y_test):
#     print(coeff_k, coeff_s)

    soil_coeff_dict = {}
    keys = range(5)
    for i in keys:
        j = i+1
        soil_coeff_dict[j] = coeff_s[i]
#     print(soil_coeff_dict)
    
    test_DF = pd.DataFrame(data=np.append(X_test, y_test.reshape(-1,1), axis=1), columns=['Crop_Type', 'RSM', 'max_RSM', 'Soil_Class', 'Field_SM'])
    test_DF['Soil_Class'] = test_DF['Soil_Class'].map(soil_coeff_dict)
#     print(test_DF)
#     print(coeff_k)
    test_DF['pred_Field_SM'] = coeff_k[0] + coeff_k[1] * test_DF['RSM']**test_DF['Soil_Class']/test_DF['max_RSM']
#     print(test_DF)
    plt.scatter(test_DF['pred_Field_SM'], test_DF['Field_SM'])

In [82]:
# basepath = 'C:\\Users\\soi\\Documents\\MTECH-PROJ-Phase-2\\'
basepath = 'C:\\Users\\rukmangadan\\Documents\\MTECH-PROJ-Phase-2\\'
# basepath = 'C:\\Users\\theorist\\Documents\\MTECH-PROJ-Phase-2\\'
in_file = basepath + 'd_soil_moisture_comparison_data.xlsx'
SM_ML_df = load_data(in_file)
SM_ML_df = replace_df_with_crop_classes_andMaptoInt(basepath, SM_ML_df)
#     print(SM_ML_df)
X_train, X_test, y_train, y_test = make_data_ready(SM_ML_df)

In [85]:
# print (X_train[X_train['Soil_Class']==3], y_train)
soil_class = 5
train_input = X_train[X_train['Soil_Class']==soil_class][['rsm_VATI', 'max_RSM']].values
train_output = X_train[X_train['Soil_Class']==soil_class][['Field_SM']].values
# print (train_input, train_output)
params, params_cov = curve_fit(soil_mapping_function, (train_input[:,0], train_input[:,1]), train_output[:,0])
print(params)

[ 4.67779436e+03 -4.38403077e+03  1.40167685e-04]
